In [ ]:

from pathlib import Path
import shutil
import zipfile

# Kaggle の入力データセット slug（username/slug の slug 部分）
DATASET_SLUG = 'gns-codes'
DATASET_ROOT = Path(f'/kaggle/input/{DATASET_SLUG}')
WORK_ROOT = Path('/kaggle/working')
repo_dir = WORK_ROOT / 'code'

code_dir = DATASET_ROOT / 'code'
code_zip = DATASET_ROOT / 'code.zip'

# 展開済み code/ を優先し、無ければ code.zip を展開
if code_dir.exists():
    src = code_dir
elif code_zip.exists():
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    with zipfile.ZipFile(code_zip) as zf:
        zf.extractall(repo_dir)
    src = repo_dir
else:
    raise FileNotFoundError(f"/kaggle/input/{DATASET_SLUG} に code も code.zip もありません。Add Data で {DATASET_SLUG} を追加してください。")

# フラット化（zip解凍で1階層挟まった場合）
if src != repo_dir:
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    shutil.copytree(src, repo_dir)
if repo_dir.exists():
    children = list(repo_dir.iterdir())
    if len(children) == 1 and children[0].is_dir():
        inner = children[0]
        for p in inner.iterdir():
            p.rename(repo_dir / p.name)
        inner.rmdir()

%cd $repo_dir


In [ ]:

# PyG の radius_graph で必要になる torch-cluster だけインストール
import torch

torch_ver = torch.__version__.split('+')[0]
cuda_ver = torch.version.cuda
cuda_tag = 'cpu' if cuda_ver is None else 'cu' + cuda_ver.replace('.', '')
url = f"https://data.pyg.org/whl/torch-{torch_ver}+{cuda_tag}.html"
print('PyG wheels:', url)

!pip -q install torch-cluster -f {url}
!pip -q install torch_geometric


In [ ]:

from pathlib import Path
import yaml

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config_rollout.yaml'

# 推論に使うデータセット（Kaggle Dataset として追加してください）
dataset_root = "/kaggle/input/tank-sloshing-mid-800"

# 出力設定
output_root = REPO / 'rollouts'
output_filename = 'inference_result'
viz_format = 'html'  # html|mp4|gif

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

cfg['method'] = 'gns'
cfg['rollout_inference_max_examples'] = 1
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

# モデルはコードに同梱された models/ から取得
model_dir = REPO / 'models'
cfg['model_path'] = str(model_dir)

import re
resolved_model_file = None
if model_dir.exists():
    expr = re.compile(r'model-(\d+)\.pt$')
    best = None
    for path in model_dir.rglob('model-*.pt'):
        m = expr.search(path.name)
        step = int(m.group(1)) if m else -1
        entry = (path.stat().st_mtime, step, path)
        if best is None or entry > best:
            best = entry
    if best:
        resolved_model_file = best[2]

if resolved_model_file is not None:
    cfg['model_file'] = str(resolved_model_file)
else:
    cfg['model_file'] = 'latest'  # models/latest を想定

cfg['output_path'] = str(output_root)
cfg['output_filename'] = output_filename

print('model_file:', cfg['model_file'])
print('output_path:', cfg['output_path'])
print('output_filename:', cfg['output_filename'])

# 設定を保存
with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)



In [ ]:

%cd /kaggle/working/code

import subprocess
from pathlib import Path

# 推論実行
cmd = ['python', 'src/train.py', '--config', 'config_rollout.yaml']
print(f"Running: {' '.join(cmd)}")
subprocess.run(cmd, check=True)

# 可視化
output_dir = Path(output_root) / 'gns' / output_filename
pkl_path = output_dir / f"{output_filename}_ex0.pkl"
viz_out = output_dir / f"{output_filename}.{viz_format}"

viz_cmd = [
    'python',
    'visualize_rollout.py',
    str(pkl_path),
    '--format', viz_format,
    '--output', str(viz_out),
]
print(f"Visualizing: {' '.join(viz_cmd)}")
subprocess.run(viz_cmd, check=True)

print(f"\n✅ Complete! Output: {viz_out}")

